In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
# !kaggle datasets download -d kwullum/fatal-police-shootings-in-the-us
!unzip fatal-police-shootings-in-the-us.zip -d .
!rm fatal-police-shootings-in-the-us.zip

Archive:  fatal-police-shootings-in-the-us.zip
  inflating: ./MedianHouseholdIncome2015.csv  
  inflating: ./PercentOver25CompletedHighSchool.csv  
  inflating: ./PercentagePeopleBelowPovertyLevel.csv  
  inflating: ./PoliceKillingsUS.csv  
  inflating: ./ShareRaceByCity.csv   


In [2]:
# I would get this error if i tried to load it as an utf-8
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf6 in position 68014: invalid start byte
with open("ShareRaceByCity.csv") as src_file:
        result =  src_file.encoding
result

'cp1252'

In [2]:
# We then have Windows-1252 formatted files

df_hh_income = pd.read_csv('MedianHouseholdIncome2015.csv', encoding='windows-1252')
df_pct_poverty = pd.read_csv('PercentagePeopleBelowPovertyLevel.csv', encoding='windows-1252')
df_pct_completed_hs = pd.read_csv('PercentOver25CompletedHighSchool.csv', encoding='windows-1252')
df_share_race_city = pd.read_csv('ShareRaceByCity.csv', encoding='windows-1252')
df_fatalities = pd.read_csv('PoliceKillingsUS.csv', encoding='windows-1252')

## Preliminary data exploration

We're going to look at the features, what are the missing values and how we could feature engineer these databases.

The explanations of the code will be written for the first dataset, and then they'll be roughly the same

### Median Household Income

In [4]:
#How many columns and rows ?
df_hh_income.shape

(29322, 3)

In [5]:
#Quick view of the features
df_hh_income.head()

,Geographic Area,City,Median Income
0,AL,Abanda CDP,11207
1,AL,Abbeville city,25615
2,AL,Adamsville city,42575
3,AL,Addison town,37083
4,AL,Akron town,21667


In [98]:
df_hh_income.describe()

,Geographic Area,City,Median Income
count,29322,29322,29271
unique,51,24249,14592
top,PA,Franklin city,(X)
freq,1762,16,1113


In [6]:
#Quick view of the values ( and check if nothing is missing )
pd.unique(df_hh_income["Geographic Area"].values)

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [7]:
pd.unique(df_hh_income["City"].values)

array(['Abanda CDP', 'Abbeville city', 'Adamsville city', ...,
       'Wright town', 'Yoder town', 'Y-O Ranch CDP'], dtype=object)

In [8]:
df_hh_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29322 entries, 0 to 29321
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Geographic Area  29322 non-null  object
 1   City             29322 non-null  object
 2   Median Income    29271 non-null  object
dtypes: object(3)
memory usage: 687.4+ KB


**Even median income has to be converted**

In [9]:
# Check missing values
df_hh_income.isnull().sum()

Geographic Area     0
City                0
Median Income      51
dtype: int64

#### Is there something else than valid numbers ?

In [10]:
# Check if there is something that is not a number in the income column
pd.unique(df_hh_income["Median Income"].str.extractall(r"([\D])").values.ravel())

array(['(', 'X', ')', '-', ',', '+'], dtype=object)

In [11]:
df_hh_income["Median Income"].str.extractall(r"([,])").head()

,,0
,match,
1347,0,","
1992,0,","
2611,0,","
2628,0,","
2885,0,","


In [12]:
df_hh_income["Median Income"].str.extractall(r"([-])").head()

,,0
,match,
71,0,-
74,0,-
84,0,-
338,0,-
462,0,-


In [13]:
# Apparently some missing data are described with (X)
df_hh_income.iloc[44]

Geographic Area             AL
City               Benton town
Median Income              (X)
Name: 44, dtype: object

In [14]:
# and we seem to have numbers with commas and +/- to indicate superior/inferior boundaries
df_hh_income.iloc[1992]

Geographic Area               CA
City               Atherton town
Median Income           250,000+
Name: 1992, dtype: object

In [15]:
# And some - are used to show null values
df_hh_income.iloc[71]

Geographic Area           AL
City               Bucks CDP
Median Income              -
Name: 71, dtype: object

### Percentage People Below Poverty Level

In [16]:
df_pct_poverty.shape

(29329, 3)

In [17]:
df_pct_poverty.head()

,Geographic Area,City,poverty_rate
0,AL,Abanda CDP,78.8
1,AL,Abbeville city,29.1
2,AL,Adamsville city,25.5
3,AL,Addison town,30.7
4,AL,Akron town,42


**We could merge the two first datasets together for better usability**

In [18]:
df_pct_poverty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Geographic Area  29329 non-null  object
 1   City             29329 non-null  object
 2   poverty_rate     29329 non-null  object
dtypes: object(3)
memory usage: 687.5+ KB


In [19]:
df_pct_poverty.isnull().sum()

Geographic Area    0
City               0
poverty_rate       0
dtype: int64

In [20]:
pd.unique(df_pct_poverty["Geographic Area"].values)

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [21]:
pd.unique(df_pct_poverty["City"].values)

array(['Abanda CDP', 'Abbeville city', 'Adamsville city', ...,
       'Wright town', 'Yoder town', 'Y-O Ranch CDP'], dtype=object)

In [22]:
pd.unique(df_pct_poverty["poverty_rate"].str.extractall(r"([\D])").values.ravel())

array(['.', '-'], dtype=object)

In [23]:
df_pct_poverty["poverty_rate"].str.extractall(r"(-)").head()

,,0
,match,
573,0,-
608,0,-
632,0,-
637,0,-
662,0,-


In [24]:
# - is used to show null values
df_pct_poverty.iloc[573]

Geographic Area             AL
City               Whatley CDP
poverty_rate                 -
Name: 573, dtype: object

### Percent Over 25 Completed HighSchool

In [14]:
df_pct_completed_hs.shape

(29329, 3)

In [15]:
df_pct_completed_hs.head()

,Geographic Area,City,percent_completed_hs
0,AL,Abanda CDP,21.2
1,AL,Abbeville city,69.1
2,AL,Adamsville city,78.9
3,AL,Addison town,81.4
4,AL,Akron town,68.6


In [16]:
df_pct_completed_hs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Geographic Area       29329 non-null  object
 1   City                  29329 non-null  object
 2   percent_completed_hs  29329 non-null  object
dtypes: object(3)
memory usage: 687.5+ KB


In [17]:
df_pct_completed_hs.isnull().sum()

Geographic Area         0
City                    0
percent_completed_hs    0
dtype: int64

In [22]:
pd.unique(df_pct_completed_hs["percent_completed_hs"].str.extractall(r"(\D)").values.ravel())

array(['.', '-'], dtype=object)

In [23]:
df_pct_completed_hs["percent_completed_hs"].str.extractall(r"(-)").head()

,,0
,match,
573,0,-
632,0,-
637,0,-
662,0,-
667,0,-


In [20]:
df_pct_completed_hs.iloc[667]

Geographic Area                    AK
City                    Edna Bay city
percent_completed_hs                -
Name: 667, dtype: object

### Share Race By City

In [44]:
df_share_race_city.shape

(29268, 7)

In [26]:
df_share_race_city.head()

,Geographic area,City,share_white,share_black,share_native_american,share_asian,share_hispanic
0,AL,Abanda CDP,67.2,30.2,0,0,1.6
1,AL,Abbeville city,54.4,41.4,0.1,1,3.1
2,AL,Adamsville city,52.3,44.9,0.5,0.3,2.3
3,AL,Addison town,99.1,0.1,0,0.1,0.4
4,AL,Akron town,13.2,86.5,0,0,0.3


In [48]:
df_share_race_city.isnull().sum()

Geographic area          0
City                     0
share_white              0
share_black              0
share_native_american    0
share_asian              0
share_hispanic           0
dtype: int64

In [47]:
pd.unique(df_share_race_city["Geographic area"].values)

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [111]:
pd.unique(df_share_race_city["share_hispanic"].str.extractall(r"([\D])").values.ravel())

array(['.'], dtype=object)

In [52]:
pd.unique(df_share_race_city["share_white"].str.extractall(r"([\D])").values.ravel())

array(['.', '(', 'X', ')'], dtype=object)

**Seems we have the same problem with (X)'s and we can suppose it's the same for the others columns**

### Police killings

In [4]:
df_fatalities.shape

(2535, 14)

In [5]:
df_fatalities.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False


Taking a look at [the shooting of Matthew Hoffman report](https://sfdistrictattorney.org/wp-content/uploads/2020/10/Hoffman-final-02142018_0.pdf), we can see that all the informations are related to the person shot, except the manner of death and the body camera who are from the Police officers "features"

In [6]:
df_fatalities.isnull().sum()

id                           0
name                         0
date                         0
manner_of_death              0
armed                        9
age                         77
gender                       0
race                       195
city                         0
state                        0
signs_of_mental_illness      0
threat_level                 0
flee                        65
body_camera                  0
dtype: int64

**We have a bunch of null values to handle**

In [56]:
df_fatalities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2535 non-null   int64  
 1   name                     2535 non-null   object 
 2   date                     2535 non-null   object 
 3   manner_of_death          2535 non-null   object 
 4   armed                    2526 non-null   object 
 5   age                      2458 non-null   float64
 6   gender                   2535 non-null   object 
 7   race                     2340 non-null   object 
 8   city                     2535 non-null   object 
 9   state                    2535 non-null   object 
 10  signs_of_mental_illness  2535 non-null   bool   
 11  threat_level             2535 non-null   object 
 12  flee                     2470 non-null   object 
 13  body_camera              2535 non-null   bool   
dtypes: bool(2), float64(1), 

In [80]:
# Checking if all the values are valid

In [73]:
pd.unique(df_fatalities.manner_of_death.values.ravel())

array(['shot', 'shot and Tasered'], dtype=object)

In [74]:
pd.unique(df_fatalities.armed.values.ravel())

array(['gun', 'unarmed', 'toy weapon', 'nail gun', 'knife', 'vehicle',
       'shovel', 'hammer', 'hatchet', 'undetermined', 'sword', 'machete',
       'box cutter', 'metal object', 'screwdriver', 'lawn mower blade',
       'flagpole', 'guns and explosives', 'cordless drill', 'crossbow',
       'metal pole', 'Taser', 'metal pipe', 'metal hand tool',
       'blunt object', 'metal stick', 'sharp object', 'meat cleaver', nan,
       'carjack', 'chain', "contractor's level", 'unknown weapon',
       'stapler', 'beer bottle', 'bean-bag gun',
       'baseball bat and fireplace poker', 'straight edge razor',
       'gun and knife', 'ax', 'brick', 'baseball bat', 'hand torch',
       'chain saw', 'garden tool', 'scissors', 'pole', 'pick-axe',
       'flashlight', 'baton', 'spear', 'pitchfork', 'hatchet and gun',
       'rock', 'piece of wood', 'bayonet', 'pipe', 'glass shard',
       'motorcycle', 'metal rake', 'crowbar', 'oar', 'machete and gun',
       'tire iron', 'air conditioner', 'pole a

**We have to combine undetermined, nan, unknown weapon into a single category : Unknown**

In [79]:
pd.unique(df_fatalities.gender.values.ravel())

array(['M', 'F'], dtype=object)

In [81]:
pd.unique(df_fatalities.threat_level.values.ravel())

array(['attack', 'other', 'undetermined'], dtype=object)

In [82]:
pd.unique(df_fatalities.flee.values.ravel())

array(['Not fleeing', 'Car', 'Foot', 'Other', nan], dtype=object)

## Data cleaning

### Median income

In [3]:
# Turn fake nan's into real ones
df_hh_income["Median Income"].replace(["(X)","-"], np.NaN, inplace=True)

In [4]:
df_hh_income.isna().sum()

Geographic Area       0
City                  0
Median Income      1904
dtype: int64

In [5]:
#Turn boundaries into constant values
df_hh_income["Median Income"].replace(regex="([,]*\d[+])", value=250000, inplace=True)

In [6]:
df_hh_income["Median Income"].replace(regex="([,]*\d[-])", value=2500, inplace=True)

In [7]:
pd.unique(df_hh_income["Median Income"].str.extractall(r"([\D])").values.ravel())

array([], dtype=object)

In [8]:
df_hh_income["Median Income"] = pd.to_numeric(df_hh_income["Median Income"])

In [9]:
# Fill nan's with median
df_hh_income["Median Income"].fillna(df_hh_income["Median Income"].median(), inplace=True)

In [10]:
df_hh_income.isna().sum()

Geographic Area    0
City               0
Median Income      0
dtype: int64

In [11]:
df_hh_income.to_csv("MedianHouseholdIncome2015_cleaned.csv", index=False)

### Percentage People Below Poverty Level

In [12]:
df_pct_poverty["poverty_rate"].replace("-",np.NaN,inplace=True)
df_pct_poverty["poverty_rate"] = pd.to_numeric(df_pct_poverty["poverty_rate"])
df_pct_poverty["poverty_rate"].fillna(df_pct_poverty["poverty_rate"].mean(),inplace=True)

In [13]:
df_pct_poverty.isnull().sum()

Geographic Area    0
City               0
poverty_rate       0
dtype: int64

In [14]:
df_pct_poverty.to_csv("PercentagePeopleBelowPovertyLevel_cleaned.csv", index=False)

## Completed highschool

In [15]:
df_pct_completed_hs.percent_completed_hs = df_pct_completed_hs.percent_completed_hs.replace("-", np.NaN)
df_pct_completed_hs.percent_completed_hs = pd.to_numeric(df_pct_completed_hs.percent_completed_hs)
df_pct_completed_hs.fillna(df_pct_completed_hs.percent_completed_hs.mean(), inplace= True)
df_pct_completed_hs.to_csv("PercentOver25CompletedHighSchool_cleaned.csv", index=False)

### Share Race By City

In [16]:
# Dans chaque share_ , replace avec lambda
df_share_race_city[["share_white","share_black","share_native_american","share_asian","share_hispanic"]] = df_share_race_city[["share_white","share_black","share_native_american","share_asian","share_hispanic"]].replace("(X)",np.NaN)
df_share_race_city[["share_white","share_black","share_native_american","share_asian","share_hispanic"]] = df_share_race_city[["share_white","share_black","share_native_american","share_asian","share_hispanic"]].apply(pd.to_numeric)
df_share_race_city["share_white"].fillna(df_share_race_city["share_white"].mean(), inplace=True)
df_share_race_city["share_black"].fillna(df_share_race_city["share_black"].mean(), inplace=True)
df_share_race_city["share_native_american"].fillna(df_share_race_city["share_native_american"].mean(), inplace=True)
df_share_race_city["share_asian"].fillna(df_share_race_city["share_asian"].mean(), inplace=True)
df_share_race_city["share_hispanic"].fillna(df_share_race_city["share_hispanic"].mean(), inplace=True)

In [17]:
df_share_race_city.to_csv("ShareRaceByCity_cleaned.csv", index=False)

## Police killings

In [3]:
df_fatalities.armed = df_fatalities.armed.replace(["undetermined","unknown weapon","unknown"], np.NaN)
df_fatalities.armed.fillna("Unknown", inplace=True)

In [4]:
df_fatalities.race.replace("O",np.NaN, inplace=True)
df_fatalities.race.fillna("U", inplace=True) #Unknown

In [5]:
df_fatalities.flee.fillna("Other",inplace=True)

In [6]:
df_fatalities.age.fillna(df_fatalities.age.mean, inplace=True)

In [7]:
df_fatalities.date = pd.to_datetime(df_fatalities.date)

In [8]:
df_fatalities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       2535 non-null   int64         
 1   name                     2535 non-null   object        
 2   date                     2535 non-null   datetime64[ns]
 3   manner_of_death          2535 non-null   object        
 4   armed                    2535 non-null   object        
 5   age                      2535 non-null   object        
 6   gender                   2535 non-null   object        
 7   race                     2535 non-null   object        
 8   city                     2535 non-null   object        
 9   state                    2535 non-null   object        
 10  signs_of_mental_illness  2535 non-null   bool          
 11  threat_level             2535 non-null   object        
 12  flee                     2535 non-

In [ ]:
df_fatalities.to_csv("PoliceKillingsUS_cleaned.csv", index=False)

# this code generates an error message but still creates the correct file